In [83]:
from sympy import *
import itertools as it
import numpy as np

In [84]:
# Make symbols: methods to dynamically create as many as needed strings and Sympy symbols
# Shift is starting index; default 0

def make_strings(N,s, shift = 0):
	my_strings = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    my_strings.append(tmp_st)
	return my_strings

def make_symbols(N,s, shift = 0):
	my_symbols = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    globals()[tmp_st] = Symbol(tmp_st)
	    my_symbols.append(globals()[tmp_st])
	return my_symbols

# Generate harmonic game

Given measure, generate harmonic game

## Set Game skeleton

In [85]:
# Game skeleton

# size of skeleton = number of players
# each entry of skeleton = number of actions of each player

skeleton = [2, 2, 2]
skeleton

[2, 2, 2]

## Set Harmonic measure

In [86]:
# Harmonic measure
# controls position of equilibrium; for default (center) set all = 1
# else can choose any strictly positive number for each entry

mu_values = [  [1, 1], [1, 2], [1,3] ]

# Size is determined by skeleton
assert len(mu_values) == len(skeleton)
for i, Ai in enumerate(skeleton):
    assert len(mu_values[i]) == Ai

mu_values

[[1, 1], [1, 2], [1, 3]]

---

In [87]:
# N
numPlayers = len(skeleton)
players = range(numPlayers)

# -------------------------
# Number of action profiles
# -------------------------

# A
numPures = prod(skeleton)

# List of A_{-i} = for each player, number of action profiles of other players
numPuresMinus = [  int(numPures / skeleton[i]) for i in players ]

# -------------------------
# Number of payoff degrees of freedom
# -------------------------

# AN; number of payoff degrees of freedom
numPays = numPlayers * numPures

# sum_i A_i; number of harmonic measure dofs
numMeas = sum(skeleton)

In [88]:
# Pure actions: list of N lists, each with Ai elements; pure actions of each player

pures_play = [ make_strings(skeleton[i], f'a{i}', shift = 1) for i in players ]

In [89]:
# Pure profiles; cartesian product of pure actions of each player
# Returns one list with A = numPures elements; each element is a tuple of strings

pures = list(it.product(*pures_play))

In [90]:
assert len(pures) == numPures

In [91]:
# Pack mu in list of dicts

mu = [   dict(zip( pures_play[i], mu_values[i] )) for i in players    ]

In [92]:
def print_payoff(payoff_dict, payoff_symbol):
    for i in players:
        for a in pures:
            print( f'{payoff_symbol}_{i}{a} = {payoff_dict[i][a]}' )
        print()

# Profiles of other players

In [93]:
# Pure profiles of other players
# Make list of N lists; the list pure_minus[i] contains the pure action profiles of players other than i
# Build taking the cartesian product of pure actions of all players other than i
# The size of the list pure_minus[i] is A_{-i} = \prod_{j \neq i} A_j

pures_minus = [ list( it.product( *( pures_play[:i] + pures_play[i+1:] ) ) ) for i in players ]

In [94]:
for i in players:
    assert len(pures_minus[i]) == numPuresMinus[i]

# Make a

In [95]:
# Util to make (a_i, a_{-i}) given a_i and a_{-i} as tuple of strings (to be used as key for payoff dictionaries)
def make_pure(ai, a_minus_i, i):
    l = list(a_minus_i)
    return tuple(l[:i] + [ai] + l[i:])

In [96]:
def split_pure(a, i):
    
    ai = a[i]

    l = list(a)
    a_minus_i = tuple( l[:i] + l[i+1:] )
    return ai, a_minus_i

In [97]:
# check consistency
for i in players:
    for a in pures:
        ai, a_minus_i = split_pure(a, i)
        assert a == make_pure( ai, a_minus_i, i )

# Harmonic payoff

In [98]:
# harmonic payoff degrees of freedom; will build harmonic payoff h out of these

# to be determined harmonic payoff of player i (as many as A)

h_sym = [make_symbols(numPures, f'h{i}', shift = 1) for i in players]


In [99]:
h = [  dict(zip(pures , h_sym[i] ))  for i in players]


# System

In [100]:
# --------------------------------
# KEY: HARMONIC FUNCTION
# --------------------------------
def Hi(a, i):
    ai, a_minus_i = split_pure(a, i)
    return sum(   [   mu[i][bi] * ( h[i][a] - h[i][ make_pure(bi, a_minus_i, i) ] )    for bi in pures_play[i]   ]  )

In [101]:
# --------------------------------
# KEY: HARMONIC SYSTEM in degrees of freedom h, that is payoff to be determined, prescribed the harmonic measure
# --------------------------------
system = [  sum( [  Hi(a,i)   for i in players ] )   for a in pures  ]

In [102]:
# one equation per pure in pures
assert len(system) == numPures

In [103]:
# Unknowns of system to solve: the weights (one per player) + the non-strategic dofs
dofs = list(it.chain(*h_sym))

In [104]:
numDofs = len(dofs)

In [105]:
# Unknowns of system to solve: harmonic weights
assert numDofs == numPays

In [106]:
# Matrix of linear system
A, b = linear_eq_to_matrix(system, *dofs)

In [107]:
sol = solve(system, dofs)

In [108]:
# Replace sol in dofs

extracted_sol = [e.subs(sol) for e in dofs]

# General solution here

In [109]:
# zip solution in dictionary with dofs
sol_dict = dict(zip(dofs, extracted_sol))

---

# Build solution instance

In [110]:
# fix remaining dofs; generate dictionary with dofs keys and random values
fix_remaining_dofs = dict(zip(dofs, np.random.randint(-5, 5, numDofs)))

In [111]:
# fix remaining dofs to random number by subbing in sol_dict
sol_dict_instance = { key : sol_dict[key].subs(fix_remaining_dofs) for key in sol_dict}

In [112]:
sol_dict_instance

{h01: 10,
 h02: -10,
 h03: -3,
 h04: 6,
 h05: 1,
 h06: -1,
 h07: 3,
 h08: 1,
 h11: 2,
 h12: -2,
 h13: 2,
 h14: -5,
 h15: 1,
 h16: -3,
 h17: -5,
 h18: 2,
 h21: -4,
 h22: -1,
 h23: 0,
 h24: -2,
 h25: 0,
 h26: 1,
 h27: -1,
 h28: -1}

# Result: harmonic game

In [115]:
skeleton

[2, 2, 2]

In [118]:
sol_dict_instance.values()

dict_values([10, -10, -3, 6, 1, -1, 3, 1, 2, -2, 2, -5, 1, -3, -5, 2, -4, -1, 0, -2, 0, 1, -1, -1])

In [119]:
numPures

8

## Order
This output give the flattened payoff vector; the first A numbers are the payoffs of the first player, the second A numbers are the payoffs of the second player, and so on.

For each player, the order in the 2x2x2 case is

```
1: (1, 1, 1)
2: (1, 1, 2)
3: (1, 2, 1)
4: (1, 2, 2)
5: (2, 1, 1)
6: (2, 1, 2)
7: (2, 2, 1)
8: (2, 2, 2)
```

# End